In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from functools import reduce, partial

from IPython.display import display, Markdown

def read_data():
    data = Path('../data/train_tickets.txt').read_text()
    sections = data.split("\n\n")
    
    ticket_fields = {}
    for line in sections[0].splitlines():
        field, ranges = line.split(":")
        ticket_fields[field] = []
        ranges = ranges.split(" or ")
        for r in ranges:
            low, high = r.split("-")
            ticket_fields[field].append((int(low),int(high)))
            
    my_ticket = [
        int(i) for i in sections[1].splitlines()[1].split(",")
    ]
    
    other_tickets = []
    for line in sections[2].splitlines()[1:]:
        other_tickets.append([
            int(i) for i in line.split(",") ])
            
    return ticket_fields, my_ticket, other_tickets

def flag_invalid(value, ranges):
    """If value is in one of the input ranges, return 0 (no flag as invalid)
    Else, return value (to add to the sum asked for by problem 1)
    """
    for r in ranges:
        if r[0] <= value <= r[1]:
            return 0
    return value

In [2]:
ticket_fields, my_ticket, other_tickets = read_data()
num_fields = len(ticket_fields)

all_ranges = []
for items in ticket_fields.values():
    all_ranges.extend(items)
    
ticket_arr = np.array(other_tickets)
invalid_fields = np.vectorize(partial(flag_invalid, ranges=all_ranges))(ticket_arr)
invalid_fields.sum()

26009

# Problem 2

Figure out which part of the ticket goes with which field and
multiply the departure fields together. 

In [3]:
# Turn invalid_fields from problem 1 into a row-mask of valid tickets
row_mask = np.sum(invalid_fields, axis=1) == 0
valid_ticket_arr = ticket_arr[row_mask, :]

In [4]:
def background(data, color='green'):
    if data.ndim == 1:
        return [ f'background-color: {color}' for _ in data ]
    else:
        new_thing = pd.DataFrame(index=data.index, columns=data.columns)
        new_thing[:,:] = f'background_color: {color}'
    return new_thing

In [5]:
# For each field, see which columns are valid using same function, but not merging all the ranges together
field_to_columns = {}
for field in ticket_fields.keys():
    invalid_fields = np.vectorize(partial(flag_invalid, ranges=ticket_fields[field]))(valid_ticket_arr)
    # vector[i] is true if the ith entry could be "field" 
    field_to_columns[field] =  np.sum(invalid_fields, axis=0) == 0

# rows = ticket entry locations, columns = names of fields
# True: haven't eliminated the possibility that this location could be that field
possible_map_df = pd.DataFrame(field_to_columns)

DEBUG = True
solution = {}
for i in range(num_fields):
    
    # find the column that only has one possible ticket index match
    # the value idxmin should = 1
    row_sums = possible_map_df.apply(np.sum, axis=0)
    field = row_sums.idxmin()
    
    # the ticket index that matches is the one true (the max)
    # in a series of falses
    index = possible_map_df[field].idxmax()
    
    solution[field] = index
    
    if DEBUG:
        # a pretty map of what we chose as an answer
        display(Markdown(f"## Iteration {i}"))
        display(possible_map_df.style
                 .apply(partial(background, color='lightyellow'), subset=pd.IndexSlice[index,:])
                 .apply(partial(background, color='lightyellow'), subset=pd.IndexSlice[:,field])
                 .apply(partial(background, color='lightcyan'), subset=pd.IndexSlice[index,field])
               )
       
    # drop the row and column from the dataframe, then we can recompute the sum
    possible_map_df = possible_map_df.drop(field, axis='columns')
    possible_map_df = possible_map_df.drop(index, axis='index')

## Iteration 0

,departure location,departure station,departure platform,departure track,departure date,departure time,arrival location,arrival station,arrival platform,arrival track,class,duration,price,route,row,seat,train,type,wagon,zone
0,False,False,False,False,False,True,False,False,False,False,True,True,False,True,False,False,True,False,False,False
1,True,True,True,True,True,True,False,False,False,False,True,True,False,True,False,False,True,False,False,True
2,True,True,True,True,True,True,False,True,False,True,True,True,False,True,False,False,True,False,True,True
3,True,True,True,True,True,True,False,False,False,False,True,True,False,True,False,False,True,False,False,False
4,True,True,True,False,False,True,False,False,False,False,True,True,False,True,False,False,True,False,False,False
5,True,True,False,False,False,True,False,False,False,False,True,True,False,True,False,False,True,False,False,False
6,True,True,True,True,True,True,False,True,False,False,True,True,False,True,False,False,True,False,True,True
7,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False,False,False,False
8,True,True,True,True,True,True,False,True,False,False,True,True,False,True,False,False,True,False,False,True
9,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False


## Iteration 1

,departure location,departure station,departure platform,departure track,departure date,departure time,arrival location,arrival station,arrival platform,arrival track,class,duration,price,route,row,seat,train,wagon,zone
0,False,False,False,False,False,True,False,False,False,False,True,True,False,True,False,False,True,False,False
1,True,True,True,True,True,True,False,False,False,False,True,True,False,True,False,False,True,False,True
2,True,True,True,True,True,True,False,True,False,True,True,True,False,True,False,False,True,True,True
3,True,True,True,True,True,True,False,False,False,False,True,True,False,True,False,False,True,False,False
4,True,True,True,False,False,True,False,False,False,False,True,True,False,True,False,False,True,False,False
5,True,True,False,False,False,True,False,False,False,False,True,True,False,True,False,False,True,False,False
6,True,True,True,True,True,True,False,True,False,False,True,True,False,True,False,False,True,True,True
7,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False,False,False
8,True,True,True,True,True,True,False,True,False,False,True,True,False,True,False,False,True,False,True
9,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False


## Iteration 2

...

,class,duration,route
7,True,True,True
9,False,False,True
11,False,True,True


## Iteration 18

,duration,route
9,False,True
11,True,True


## Iteration 19

,route
9,True


In [6]:
row_sums

route    1
dtype: int64

In [7]:
indexes = [solution[field] for field in solution.keys() if field.startswith('departure')]
values = [my_ticket[i] for i in indexes ] 
prod = reduce(lambda a,b:a*b, values, 1)
prod

589685618167